# 텍스트 데이터 처리 기초 예시


---

# 1. 짧은 3문장 처리
> i feel hungry<br>
> i eat lunch<br>
> now i feel happy


### 1-1. 우선 문장을 모두 list에 담기

In [1]:
# 처리해야 할 문장을 list에 담기
sentences=['i feel hungry', 'i eat lunch', 'now i feel happy']

# 파이썬 split() 메소드를 이용해 단어 단위로 문장을 쪼개는 예시
word_list = 'i feel hungry'.split()
print(word_list)

['i', 'feel', 'hungry']


### 1-2. 단어 사전 만들기
> * 0 = PAD : PADDING용
> * 1 = BOS : Begin Of Sentence. 문장의 시작지점을 의미
> * 2 = UNK : 사전에 없는 단어를 의미

In [2]:
index_to_word={}  # 빈 딕셔너리를 만들어서

# 단어들을 하나씩 채워 봅니다. 채우는 순서는 일단 임의로 하였습니다. 그러나 사실 순서는 중요하지 않습니다. 
# <BOS>, <PAD>, <UNK>는 관례적으로 딕셔너리 맨 앞에 넣어줍니다. 
index_to_word[0]='<PAD>'  # 패딩용 단어
index_to_word[1]='<BOS>'  # 문장의 시작지점, 이 토큰을 써서 시작지점을 구분함. Begin Of Sentence
index_to_word[2]='<UNK>'  # 사전에 없는(Unknown) 단어
index_to_word[3]='i'
index_to_word[4]='feel'
index_to_word[5]='hungry'
index_to_word[6]='eat'
index_to_word[7]='lunch'
index_to_word[8]='now'
index_to_word[9]='happy'

print(index_to_word)

{0: '<PAD>', 1: '<BOS>', 2: '<UNK>', 3: 'i', 4: 'feel', 5: 'hungry', 6: 'eat', 7: 'lunch', 8: 'now', 9: 'happy'}


### 1-3. {텍스트:인덱스} 구조로 사전에 넣기
텍스트를 숫자로 바꾸려면 위의 딕셔너리가 {텍스트:인덱스} 구조여야 함.

In [3]:
word_to_index={word:index for index, word in index_to_word.items()}
print(word_to_index)

{'<PAD>': 0, '<BOS>': 1, '<UNK>': 2, 'i': 3, 'feel': 4, 'hungry': 5, 'eat': 6, 'lunch': 7, 'now': 8, 'happy': 9}


In [4]:
print(word_to_index['feel'])  # 단어 'feel'은 숫자 인덱스 4로 바뀝니다.

# 단어를 주면 인덱스로 반환하는 방식으로도 사용 가능

4


### 1-4. 문장을 받아 인덱스가 담긴 리스트로 변환해 주는 매핑 함수 생성

In [5]:
# 문장 1개를 활용할 딕셔너리와 함께 주면, 단어 인덱스 리스트로 변환해 주는 함수.
# 단, 모든 문장을 <BOS>로 시작하게 만들 것. 
def get_encoded_sentence(sentence, word_to_index):
    return [word_to_index['<BOS>']]+[word_to_index[word] if word in word_to_index else word_to_index['<UNK>'] for word in sentence.split()]

In [6]:
print(get_encoded_sentence('i eat lunch', word_to_index))

# 예시

[1, 3, 6, 7]


#### 1-4-2. 여러 개의 문장을 한꺼번에 매핑하는 함수

In [7]:
# 여러 개의 문장 리스트를 한꺼번에 숫자 텐서로 encode해 주는 함수. 
def get_encoded_sentences(sentences, word_to_index):
    return [get_encoded_sentence(sentence, word_to_index) for sentence in sentences]

In [8]:
encoded_sentences = get_encoded_sentences(sentences, word_to_index)
print(encoded_sentences)

# sentences=['i feel hungry', 'i eat lunch', 'now i feel happy'] 를 변환

[[1, 3, 4, 5], [1, 3, 6, 7], [1, 8, 3, 4, 9]]


### 1-5. 반대로 인덱스 리스트를 받아 문장으로 decode해주는 함수 생성

In [9]:
# 숫자 벡터로 encode된 문장을 원래대로 decode하는 함수. 
def get_decoded_sentence(encoded_sentence, index_to_word):
    return ' '.join(index_to_word[index] if index in index_to_word else '<UNK>' for index in encoded_sentence[1:])  #[1:]를 통해 <BOS>를 제외

In [10]:
print(get_decoded_sentence([1, 3, 4, 5], index_to_word))

i feel hungry


#### 1-5-2. 여러 개의 문장을 한꺼번에 decode하는 함수

In [11]:
# 여러 개의 숫자 벡터로 encode된 문장을 한꺼번에 원래대로 decode하는 함수입니다. 
def get_decoded_sentences(encoded_sentences, index_to_word):
    return [get_decoded_sentence(encoded_sentence, index_to_word) for encoded_sentence in encoded_sentences]

In [12]:
print(get_decoded_sentences(encoded_sentences, index_to_word))

# 위에서 encoding한 encoded_sentences=[[1, 3, 4, 5], [1, 3, 6, 7], [1, 8, 3, 4, 9]] 를 디코딩할 것

['i feel hungry', 'i eat lunch', 'now i feel happy']


---
## 2. Embedding Layer 적용 버전
<span style="color:red"> 하지만 'i feel hungry'를 1, 3, 4, 5 로 변환한 것은 임의로 부여한 순서에 맞췄을 뿐이다. </span><br>
##### 따라서 Embedding Layer를 적용하여 의미 벡터 파라미터를 구현한 버전을 확인할 것.

### 2-0. 일단 위에서 만든 사전을 이용해 다른 전처리 없이 문장을 임베딩

In [13]:
# 아래 코드는 그대로 실행하면 에러가 발생함. 

import numpy as np
import tensorflow as tf
import os

vocab_size = len(word_to_index)  # 위 예시에서 딕셔너리에 포함된 단어 개수는 10
word_vector_dim = 4    # 위 그림과 같이 4차원의 워드 벡터를 가정. 

embedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=word_vector_dim, mask_zero=True)

# 숫자로 변환된 텍스트 데이터 [[1, 3, 4, 5], [1, 3, 6, 7], [1, 8, 3, 4, 9]] 에 Embedding 레이어를 적용. 
raw_inputs = np.array(get_encoded_sentences(sentences, word_to_index), dtype='object')
output = embedding(raw_inputs)
print(output)

Metal device set to: Apple M2


2022-09-19 21:29:38.925997: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-19 21:29:38.926079: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).

<span style="color:red">에러가 발생하는 이유는 Embedding Layer의 input이 일정한 길이가 아니었기 때문.</span>

### 2-1. 전처리 : Padding을 추가하여 input의 길이를 일정하게 만들기

In [14]:
raw_inputs = np.array(get_encoded_sentences(sentences, word_to_index), dtype='object')

raw_inputs = tf.keras.preprocessing.sequence.pad_sequences(raw_inputs,
                                                       value=word_to_index['<PAD>'],
                                                       padding='post',
                                                       maxlen=5)
print(raw_inputs)

# tf.keras.preprocessing.sequence.pad_sequences를 통해 word vector를 모두 일정 길이로 맞춰주어야 
# embedding 레이어의 input이 될 수 있음. 

[[1 3 4 5 0]
 [1 3 6 7 0]
 [1 8 3 4 9]]


문장의 길이가 부족한 곳에 Padding(PAD : 0)이 채워지는 걸 확인 가능<br><br>


### 2-2. Embedding Layer에 input을 넣기

In [15]:
vocab_size = len(word_to_index)  # 위 예시에서 딕셔너리에 포함된 단어 개수는 10
word_vector_dim = 4    # 그림과 같이 4차원의 워드 벡터를 가정.

embedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=word_vector_dim, mask_zero=True)

output = embedding(raw_inputs)
print(output)

tf.Tensor(
[[[-0.04321151 -0.02590908 -0.00490627  0.03732581]
  [-0.04665475 -0.01735619  0.01745808 -0.02937414]
  [-0.01623361 -0.00493796 -0.04827733  0.03078593]
  [-0.04476795 -0.04711833 -0.0162771   0.02076251]
  [ 0.01357362  0.03454966  0.01479269 -0.03468297]]

 [[-0.04321151 -0.02590908 -0.00490627  0.03732581]
  [-0.04665475 -0.01735619  0.01745808 -0.02937414]
  [-0.0207108   0.03367747 -0.02921149 -0.03963925]
  [ 0.02684183 -0.03859163 -0.00656774  0.00030935]
  [ 0.01357362  0.03454966  0.01479269 -0.03468297]]

 [[-0.04321151 -0.02590908 -0.00490627  0.03732581]
  [ 0.04396858 -0.01116291 -0.01549724 -0.010341  ]
  [-0.04665475 -0.01735619  0.01745808 -0.02937414]
  [-0.01623361 -0.00493796 -0.04827733  0.03078593]
  [ 0.02353993 -0.00343392 -0.01920385  0.02050158]]], shape=(3, 5, 4), dtype=float32)


결과로 나오는 shape=(3, 5, 4)는 각각 입력문장 개수, 입력문장의 최대 길이, 워드 벡터의 차원 수를 의미.<br><br>


### 2-3. RNN 모델 설계
> 참고 : [RNN 강의 유튜브](https://www.youtube.com/watch?v=-SHPG_KMUkQ)

In [16]:
vocab_size = 10  # 어휘 사전의 크기(10개의 단어)
word_vector_dim = 4  # 단어 하나를 표현하는 임베딩 벡터의 차원수. 

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model.add(tf.keras.layers.LSTM(8))   # LSTM 레이어를 사용함. 이때 LSTM state 벡터의 차원수는 8로 지정하였음. (변경 가능)
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim으로.

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 4)           40        
                                                                 
 lstm (LSTM)                 (None, 8)                 416       
                                                                 
 dense (Dense)               (None, 8)                 72        
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                                 
Total params: 537
Trainable params: 537
Non-trainable params: 0
_________________________________________________________________


---

# 번외. CNN을 이용한 텍스트 처리 모델
1-D CNN은 문장 전체를 한꺼번에 한 방향으로 길이 7짜리 필터로 스캐닝 하면서 7단어 이내에서 발견되는 특징을 추출하여 그것으로 문장을 분류하는 방식으로 사용
병렬 처리에 효율적이기 때문에 학습 속도가 더 빠르다는 장점을 가짐

In [17]:
vocab_size = 10  # 어휘 사전의 크기(10개의 단어)
word_vector_dim = 4   # 단어 하나를 표현하는 임베딩 벡터의 차원 수. 

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model.add(tf.keras.layers.Conv1D(16, 7, activation='relu'))
model.add(tf.keras.layers.MaxPooling1D(5))
model.add(tf.keras.layers.Conv1D(16, 7, activation='relu'))
model.add(tf.keras.layers.GlobalMaxPooling1D())
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim 입니다.

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 4)           40        
                                                                 
 conv1d (Conv1D)             (None, None, 16)          464       
                                                                 
 max_pooling1d (MaxPooling1D  (None, None, 16)         0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, None, 16)          1808      
                                                                 
 global_max_pooling1d (Globa  (None, 16)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense_2 (Dense)             (None, 8)                

 * ### 간단히 GlobalMaxPooling1D()레이어 하나만 사용하는 모델
전체 문장 중에서 단 하나의 가장 중요한 단어만 피처로 추출하여 그것으로 문장의 긍정/부정을 평가하는 방식.

In [18]:
vocab_size = 10  # 어휘 사전의 크기입니다(10개의 단어)
word_vector_dim = 4   # 단어 하나를 표현하는 임베딩 벡터의 차원 수입니다. 

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model.add(tf.keras.layers.GlobalMaxPooling1D())
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim 입니다.

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, None, 4)           40        
                                                                 
 global_max_pooling1d_1 (Glo  (None, 4)                0         
 balMaxPooling1D)                                                
                                                                 
 dense_4 (Dense)             (None, 8)                 40        
                                                                 
 dense_5 (Dense)             (None, 1)                 9         
                                                                 
Total params: 89
Trainable params: 89
Non-trainable params: 0
_________________________________________________________________
